# Importing the required lib 


In [10]:
import pandas as pd
import sklearn
#from prophet import Prophet
#from prophet.plot import add_changepoints_to_plot

# Reading the data and exploring the data for daily

In [19]:
df = pd.read_csv('SN_d_tot_V2.0.csv', delimiter=";")
df.head()

,year,month,day,decimaldate,dsn,sd,noo,dpi
0,1818,1,1,1818.001,-1,-1.0,0,1
1,1818,1,2,1818.004,-1,-1.0,0,1
2,1818,1,3,1818.007,-1,-1.0,0,1
3,1818,1,4,1818.010,-1,-1.0,0,1
4,1818,1,5,1818.012,-1,-1.0,0,1


In [13]:
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

In [18]:
df.head()

,year,month,day,decimaldate,dsn,sd,noo,dpi,date
0,1818,1,1,1818.001,-1,-1.0,0,1,1818-01-01
1,1818,1,2,1818.004,-1,-1.0,0,1,1818-01-02
2,1818,1,3,1818.007,-1,-1.0,0,1,1818-01-03
3,1818,1,4,1818.010,-1,-1.0,0,1,1818-01-04
4,1818,1,5,1818.012,-1,-1.0,0,1,1818-01-05


In [15]:
dfslim = df[['date', 'dsn']]
dfslim = dfslim.rename(columns={'date':'ds', 'dsn': 'y'})
dfslim[:20]

,ds,y
0,1818-01-01,-1
1,1818-01-02,-1
2,1818-01-03,-1
3,1818-01-04,-1
4,1818-01-05,-1
5,1818-01-06,-1
6,1818-01-07,-1
7,1818-01-08,65
8,1818-01-09,-1
9,1818-01-10,-1


In [16]:
m = Prophet(daily_seasonality=True)
m.fit(dfslim)

NameError: name 'Prophet' is not defined

In [ ]:
future_daily_100 = m.make_future_dataframe(periods=100) # 200, # 365
future_daily_200 = m.make_future_dataframe(periods=200) # 200, # 365
future_daily_365 = m.make_future_dataframe(periods=365) # 200, # 365

In [ ]:
print(future_daily_100.tail())
print(future_daily_200.tail())
print(future_daily_365.tail())

In [ ]:
forecast100 = m.predict(future_daily_100)
forecast200 = m.predict(future_daily_200)
forecast365 = m.predict(future_daily_365)

In [ ]:
forecast100[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast200[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast365[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig100 = m.plot(forecast100)

In [ ]:
fig200 = m.plot(forecast200)

In [ ]:
fig365 = m.plot(forecast365)

In [24]:
cap = max(df['dsn'])

In [22]:
dfslim_cap = df[['date', 'dsn', 'cap']]
dfslim_cap = dfslim_cap.rename(columns={'date':'ds', 'dsn': 'y', 'cap': 'cap'})
dfslim_cap.head()
# dfslim_cap = df.insert(2, "cap", cap)
# dfslim_cap[:100]

KeyError: "['date', 'cap'] not in index"

In [ ]:
# tabular display of actual and predicted for 100 , 200 and 365 
actual_predicted100 = pd.DataFrame(zip(dfslim["y"][:465], forecast100["yhat"]), columns=["actual", "predicted"])
actual_predicted.head(100)

actual_predicted200 = pd.DataFrame(zip(dfslim["y"][:465], forecast200["yhat"]), columns=["actual", "predicted"])
actual_predicted.head(200)

actual_predicted100 = pd.DataFrame(zip(dfslim["y"][:465], forecast365["yhat"]), columns=["actual", "predicted"])
actual_predicted.head(365)

# for 100 days

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic')
m.fit(dfslim_cap[:100])

In [ ]:
future_daily_100_logistic = m.make_future_dataframe(periods=100)
future_daily_100_logistic['cap'] = cap
forceast100_logistic = m.predict(future_daily_100_logistic)
fig1 = m.plot(forceast100_logistic)

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast100_seasonality = m.fit(dfslim_cap[:100]).predict(future_daily_100)
fig1 = m.plot_components(forceast100_seasonality)

In [ ]:
fig1 = m.plot(forceast100_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast100_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:100]).predict(future_daily_100)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:100]).predict(future_daily_100)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# for 200 days

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:200])

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast200_seasonality = m.fit(dfslim_cap[:200]).predict(future_daily_200)
fig2 = m.plot_components(forceast200_seasonality)

In [ ]:
fig2 = m.plot(forceast200_seasonality)
a = add_changepoints_to_plot(fig2.gca(), m, forceast200_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:200]).predict(future_daily_200)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:200]).predict(future_daily_200)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# need to add MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# for 365 days 

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:365])

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast365_seasonality = m.fit(dfslim_cap[:365]).predict(future_daily_365)
fig3 = m.plot_components(forceast365_seasonality)

In [ ]:
fig3 = m.plot(forceast365_seasonality)
a = add_changepoints_to_plot(fig3.gca(), m, forceast365_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:365]).predict(future_daily_365)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:365]).predict(future_daily_365)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# Reading the data and exploring the data for Monthly Data

In [43]:
df = pd.read_csv('SN_m_tot_V2.0.csv', delimiter=";")
df['day'] = '1'

0       1749
1       1749
2       1749
3       1749
4       1749
        ... 
3261    2020
3262    2020
3263    2021
3264    2021
3265    2021
Name: 1749, Length: 3266, dtype: int64

In [44]:
df['date'] = pd.to_datetime(df[df.columns[0]] + df[df.columns[1]] + df["day"] # no date, need default date

SyntaxError: unexpected EOF while parsing (<ipython-input-44-e4f698387840>, line 2)

In [ ]:
df.head()

In [ ]:
dfslim = df[['date', 'dsn']]
dfslim = dfslim.rename(columns={'date':'ds', 'dsn': 'y'})
dfslim[:20]

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(dfslim[:1])

In [ ]:
future_monthly_1 = m.make_future_dataframe(periods=1) 
future_monthly_6 = m.make_future_dataframe(periods=6)
future_monthly_9 = m.make_future_dataframe(periods=9)

In [ ]:
print(future_monthly_1.tail())
print(future_monthly_6.tail())
print(future_monthly_9.tail())

In [ ]:
forecast1 = m.predict(future_monthly_1)
forecast6 = m.predict(future_monthly_6)
forecast9 = m.predict(future_monthly_9)

In [ ]:
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast6[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast9[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast1)

In [ ]:
fig6 = m.plot(forecast6)

In [ ]:
fig9 = m.plot(forecast9)

In [ ]:
cap = max(df['dsn'])

In [ ]:
dfslim_cap = df[['date', 'dsn', 'cap']]
dfslim_cap = dfslim_cap.rename(columns={'date':'ds', 'dsn': 'y', 'cap': 'cap'})
dfslim_cap.head()
# dfslim_cap = df.insert(2, "cap", cap)
# dfslim_cap[:1]

# for 1 month

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') #lineaer # flat
m.fit(dfslim_cap[:1])

In [ ]:
future_monthly_1_logistic = m.make_future_dataframe(periods=1)
future_monthly_1_logistic['cap'] = cap
forceast1_logistic = m.predict(future_monthly_1_logistic)
fig1 = m.plot(forceast1_logistic)

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast1_seasonality = m.fit(dfslim_cap[:1]).predict(future_monthly_1)
fig1 = m.plot_components(forceast1_seasonality)

In [ ]:
fig1 = m.plot(forceast1_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast1_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:1]).predict(future_monthly_1)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:1]).predict(future_monthly_1)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# for 6 month

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:6])

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast6_seasonality = m.fit(dfslim_cap[:6]).predict(future_monthly_6)
fig1 = m.plot_components(forceast6_seasonality)

In [ ]:
fig1 = m.plot(forceast6_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast6_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:6]).predict(future_monthly_6)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:6]).predict(future_monthly_6)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# for 9 month

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:9])

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast9_seasonality = m.fit(dfslim_cap[:9]).predict(future_monthly_9)
fig1 = m.plot_components(forceast9_seasonality)

In [ ]:
fig1 = m.plot(forceast9_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast9_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:9]).predict(future_monthly_9)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:9]).predict(future_monthly_9)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# Reading the data and exploring the data for yearly Data

In [13]:
df = pd.read_csv('SN_y_tot_V2.0.csv', delimiter=";")
df.head()

,1700.5,8.3,-1.0,-1,1
0,1701.5,18.3,-1.0,-1,1
1,1702.5,26.7,-1.0,-1,1
2,1703.5,38.3,-1.0,-1,1
3,1704.5,60.0,-1.0,-1,1
4,1705.5,96.7,-1.0,-1,1


In [15]:
df['date'] = pd.to_datetime(df['1700.5']) # is there any better way? date looks weird af, maybe change date format

In [16]:
df.head()

,1700.5,8.3,-1.0,-1,1,date
0,1701.5,18.3,-1.0,-1,1,1970-01-01 00:00:00.000001701
1,1702.5,26.7,-1.0,-1,1,1970-01-01 00:00:00.000001702
2,1703.5,38.3,-1.0,-1,1,1970-01-01 00:00:00.000001703
3,1704.5,60.0,-1.0,-1,1,1970-01-01 00:00:00.000001704
4,1705.5,96.7,-1.0,-1,1,1970-01-01 00:00:00.000001705


In [26]:
dfslim = df[['date', df.iloc[2]]
dfslim = dfslim.rename(columns={'date':'ds', df.iloc[2]: 'y'})
dfslim[:20]

SyntaxError: invalid syntax (<ipython-input-26-10fae52a50e9>, line 2)

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(dfslim[:1])

In [ ]:
future_yearly_1 = m.make_future_dataframe(periods=1) 
future_yearly_10 = m.make_future_dataframe(periods=10)
future_yearly_20 = m.make_future_dataframe(periods=20)

In [ ]:
print(future_yearly_1.tail())
print(future_yearly_10.tail())
print(future_yearly_20.tail())

In [ ]:
forecast1 = m.predict(future_yearly_1)
forecast10 = m.predict(future_yearly_10)
forecast20 = m.predict(future_yearly_20)

In [ ]:
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast10[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast20[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast1)

In [ ]:
fig10 = m.plot(forecast10)

In [ ]:
fig20 = m.plot(forecast20)

In [ ]:
cap = max(df['dsn'])

In [ ]:
dfslim_cap = df[['date', 'dsn', 'cap']]
dfslim_cap = dfslim_cap.rename(columns={'date':'ds', 'dsn': 'y', 'cap': 'cap'})
dfslim_cap.head()
# dfslim_cap = df.insert(2, "cap", cap)
# dfslim_cap[:1]

# for 1 Year

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:1])

In [ ]:
future_yearly_1_logistic = m.make_future_dataframe(periods=1)
future_yearly_1_logistic['cap'] = cap
forceast1_logistic = m.predict(future_yearly_1_logistic)
fig1 = m.plot(forceast1_logistic)

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast1_seasonality = m.fit(dfslim_cap[:1]).predict(future_yearly_1)
fig1 = m.plot_components(forceast1_seasonality)

In [ ]:
fig1 = m.plot(forceast1_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast1_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:1]).predict(future_yearly_1)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:1]).predict(future_yearly_1)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# for 10 Year

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:10])

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast10_seasonality = m.fit(dfslim_cap[:10]).predict(future_yearly_10)
fig1 = m.plot_components(forceast10_seasonality)

In [ ]:
fig1 = m.plot(forceast10_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast10_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:10]).predict(future_yearly_10)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:10]).predict(future_yearly_10)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations

In [ ]:
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])

# for 20 Year

In [ ]:
m = Prophet(daily_seasonality=True, growth='logistic') 
m.fit(dfslim_cap[:20])

In [ ]:
m = Prophet(daily_seasonality=True)
m.add_seasonality(name='daily', period=1, fourier_order=2)
forceast20_seasonality = m.fit(dfslim_cap[:20]).predict(future_yearly_20)
fig1 = m.plot_components(forceast20_seasonality)

In [ ]:
fig1 = m.plot(forceast20_seasonality)
a = add_changepoints_to_plot(fig1.gca(), m, forceast20_seasonality)

In [ ]:
m.changepoints

In [ ]:
m_changepoints = Prophet(daily_seasonality=True, changepoints=m.changepoints)
forecast_changepoints = m_changepoints.fit(dfslim_cap[:20]).predict(future_yearly_20)
m_changepoints.plot(forecast_changepoints);

In [ ]:
m_changepoint_prior_scale = Prophet(daily_seasonality=True, changepoint_prior_scale=0.5)
forecast_changepoint_prior_scale = m_changepoint_prior_scale.fit(dfslim_cap[:20]).predict(future_yearly_20)
m_changepoint_prior_scale.plot(forecast_changepoint_prior_scale);

In [ ]:
# MAE MAPE r2 equations
MAE = sklearn.metrics.mean_absolute_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
MAPE = sklearn.metrics.mean_absolute_percentage_error(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])
R_2 = sklearn.metrics.r2_score(dfslim_cap["y"][100:200], forecast_changepoint_prior_scale['yhat'][100:200])